# Phase 2 — Business-Aware Enrichment (Funneling & Smurfing)

In [7]:

from neo4j import GraphDatabase
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance

NEO4J_BOLT_URI = "bolt://192.168.0.5:7687"
NEO4J_AUTH     = ("neo4j", "PotatoDTND12!")
driver = GraphDatabase.driver(NEO4J_BOLT_URI, auth=NEO4J_AUTH)

APRIL_START_EPOCH = 1743476400
APRIL_END_EPOCH   = 1746068400
APRIL1_START      = 1743476400
APRIL1_END        = 1743562800

print("Setup OK.")


Setup OK.


In [8]:

def _run_single(s, q, **params):
    res = s.run(q, **params)
    rec = res.single()
    return rec.data() if rec else None

def _default_stats_dict():
    return {"n_total": 0, "mean_score": None, "std_score": None, "anomaly_rate": 0.0,
            "anom_min": None, "anom_max": None, "norm_min": None, "norm_max": None}

def one_minus_ws(df_scores):
    if df_scores.empty or df_scores["label"].nunique() < 2:
        return {"one_minus_ws": np.nan, "ws": np.nan, "clarity": "unclear"}
    a = df_scores.loc[df_scores.label==1, "score"].astype(float).values
    b = df_scores.loc[df_scores.label==0, "score"].astype(float).values
    if len(a)==0 or len(b)==0:
        return {"one_minus_ws": np.nan, "ws": np.nan, "clarity": "unclear"}
    ws = wasserstein_distance(a, b)
    lo = float(np.nanmin(np.concatenate([a,b])))
    hi = float(np.nanmax(np.concatenate([a,b])))
    rng = max(hi - lo, 1e-12)
    ws_norm = ws / rng
    sim = 1.0 - max(0.0, min(1.0, ws_norm))
    clarity = "clear" if ws_norm >= 0.2 else "unclear"
    return {"one_minus_ws": sim, "ws": ws_norm, "clarity": clarity}

def sci(x, sig=6):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return "n/a"
    if x == 0:
        return f"{0:.{sig}f} × 10^0"
    exp = int(np.floor(np.log10(abs(x))))
    mant = x / (10**exp)
    return f"{mant:.{sig}f} × 10^{exp}"


In [9]:

def stats_global_phase1(start_epoch, end_epoch):
    q = """
MATCH (tx:Transaction)
WHERE tx.timestamp >= $startEpoch AND tx.timestamp < $endEpoch
  AND tx.label IS NOT NULL AND tx.scoring IS NOT NULL
RETURN
  count(tx)                                                 AS n_total,
  avg(toFloat(tx.scoring))                                  AS mean_score,
  stDev(toFloat(tx.scoring))                                AS std_score,
  toFloat(sum(CASE WHEN toInteger(tx.label)=1 THEN 1 ELSE 0 END))/count(tx) AS anomaly_rate,
  min(CASE WHEN toInteger(tx.label)=1 THEN toFloat(tx.scoring) END)         AS anom_min,
  max(CASE WHEN toInteger(tx.label)=1 THEN toFloat(tx.scoring) END)         AS anom_max,
  min(CASE WHEN toInteger(tx.label)=0 THEN toFloat(tx.scoring) END)         AS norm_min,
  max(CASE WHEN toInteger(tx.label)=0 THEN toFloat(tx.scoring) END)         AS norm_max
"""
    with driver.session() as s:
        rec = _run_single(s, q, startEpoch=start_epoch, endEpoch=end_epoch)
    return rec or _default_stats_dict()

def stats_global_phase2(start_epoch, end_epoch):
    q = """
MATCH (tx:Transaction)
WHERE tx.timestamp >= $startEpoch AND tx.timestamp < $endEpoch
  AND tx.label_v2 IS NOT NULL AND tx.scoring_v2 IS NOT NULL
RETURN
  count(tx)                                                  AS n_total,
  avg(toFloat(tx.scoring_v2))                                AS mean_score,
  stDev(toFloat(tx.scoring_v2))                              AS std_score,
  toFloat(sum(CASE WHEN toInteger(tx.label_v2)=1 THEN 1 ELSE 0 END))/count(tx) AS anomaly_rate,
  min(CASE WHEN toInteger(tx.label_v2)=1 THEN toFloat(tx.scoring_v2) END)       AS anom_min,
  max(CASE WHEN toInteger(tx.label_v2)=1 THEN toFloat(tx.scoring_v2) END)       AS anom_max,
  min(CASE WHEN toInteger(tx.label_v2)=0 THEN toFloat(tx.scoring_v2) END)       AS norm_min,
  max(CASE WHEN toInteger(tx.label_v2)=0 THEN toFloat(tx.scoring_v2) END)       AS norm_max
"""
    with driver.session() as s:
        rec = _run_single(s, q, startEpoch=start_epoch, endEpoch=end_epoch)
    return rec or _default_stats_dict()

def sample_scores_global_phase1(start_epoch, end_epoch, limit_per_class=40000):
    q = """
MATCH (tx:Transaction)
WHERE tx.timestamp >= $startEpoch AND tx.timestamp < $endEpoch
  AND tx.label IS NOT NULL AND tx.scoring IS NOT NULL
WITH toInteger(tx.label) AS y, toFloat(tx.scoring) AS s
ORDER BY rand()
WITH y, collect(s)[0..$cap] AS ss
RETURN y AS label, ss AS scores
"""
    rows = []
    cap = int(limit_per_class)
    with driver.session() as s:
        for r in s.run(q, startEpoch=start_epoch, endEpoch=end_epoch, cap=cap):
            rows.append((r["label"], r["scores"]))
    data = []
    for label, scores in rows:
        for sc in scores:
            data.append({"label": int(label), "score": float(sc)})
    return pd.DataFrame(data)

def sample_scores_global_phase2(start_epoch, end_epoch, limit_per_class=40000):
    q = """
MATCH (tx:Transaction)
WHERE tx.timestamp >= $startEpoch AND tx.timestamp < $endEpoch
  AND tx.label_v2 IS NOT NULL AND tx.scoring_v2 IS NOT NULL
WITH toInteger(tx.label_v2) AS y, toFloat(tx.scoring_v2) AS s
ORDER BY rand()
WITH y, collect(s)[0..$cap] AS ss
RETURN y AS label, ss AS scores
"""
    rows = []
    cap = int(limit_per_class)
    with driver.session() as s:
        for r in s.run(q, startEpoch=start_epoch, endEpoch=end_epoch, cap=cap):
            rows.append((r["label"], r["scores"]))
    data = []
    for label, scores in rows:
        for sc in scores:
            data.append({"label": int(label), "score": float(sc)})
    return pd.DataFrame(data)


In [10]:

# Run
p1_apr1_stats = stats_global_phase1(APRIL1_START, APRIL1_END)
p1_apr_stats  = stats_global_phase1(APRIL_START_EPOCH, APRIL_END_EPOCH)
p1_apr1_samp  = sample_scores_global_phase1(APRIL1_START, APRIL1_END)
p1_apr_samp   = sample_scores_global_phase1(APRIL_START_EPOCH, APRIL_END_EPOCH)
p1_apr1_sim   = one_minus_ws(p1_apr1_samp)
p1_apr_sim    = one_minus_ws(p1_apr_samp)

p2_apr1_stats = stats_global_phase2(APRIL1_START, APRIL1_END)
p2_apr_stats  = stats_global_phase2(APRIL_START_EPOCH, APRIL_END_EPOCH)
p2_apr1_samp  = sample_scores_global_phase2(APRIL1_START, APRIL1_END)
p2_apr_samp   = sample_scores_global_phase2(APRIL_START_EPOCH, APRIL_END_EPOCH)
p2_apr1_sim   = one_minus_ws(p2_apr1_samp)
p2_apr_sim    = one_minus_ws(p2_apr_samp)

print("Phase 2 global comparisons ready.")


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '\nMATCH (tx:Transaction)\nWHERE tx.timestamp >= $startEpoch AND tx.timestamp < $endEpoch\n  AND tx.label IS NOT NULL AND tx.scoring IS NOT NULL\nRETURN\n  count(tx)                                                 AS n_total,\n  avg(toFloat(tx.scoring))                                  AS mean_score,\n  stDev(toFloat(tx.scoring))                                AS std_score,\n  toFloat(sum(CASE WHEN toInteger(tx.label)=1 THEN 1 ELSE 0 END))/count(tx) AS anomaly_rate,\n  min(CASE WHEN toInteger(tx.label)=1 THEN toFloat(tx.scoring) END)         AS anom_min,\n  max(CASE WHEN toInteger(tx.label)=1 THEN toFloat(tx.scoring) END)         AS anom_max,\n  min(CASE WHEN toInteger(tx.label)=0 THE

Phase 2 global comparisons ready.


In [11]:

# Word switches (based on WS change and absolute WS in Phase 2)
def switches(p1_sim, p2_sim):
    clarity = "clear" if p2_sim["ws"] >= 0.2 else "unclear"
    profiles = "distinct" if p2_sim["ws"] >= 0.2 else "indistinct"
    delta = (p2_sim["ws"] - p1_sim["ws"]) if (p1_sim["ws"] is not np.nan and p2_sim["ws"] is not np.nan) else 0.0
    benefit = "flagged" if delta >= 0.02 else "did not benefit"
    support = "supporting" if delta >= 0.02 else "unsupporting"
    return clarity, profiles, benefit, support, delta

apr1_clarity, apr1_profiles, apr1_benefit, apr1_support, apr1_delta = switches(p1_apr1_sim, p2_apr1_sim)
apr_clarity,  apr_profiles,  apr_benefit,  apr_support,  apr_delta  = switches(p1_apr_sim,  p2_apr_sim)

text = f"""Nevertheless, we started a new enrichment phase which added funneling and smurfing signaling features over all the April 2025 traffic, revealed a {apr_clarity} separation between anomaly and normal scores. As Table 4-10 shows, cases tied to these typologies began resulting in {apr_profiles} score profiles. These behaviors were {apr_benefit} from the enriched features capturing graph topology asymmetries (in degree vs. out degree) and temporal density (transaction bursts) around nodes.

Also, as shown in Table 4-11, the 1−WS similarity changed from {1.0 - p1_apr_sim['ws']:.4f} to {1.0 - p2_apr_sim['ws']:.4f} (WS distance from {p1_apr_sim['ws']:.4f} to {p2_apr_sim['ws']:.4f}), {apr_support} the intuition that business aware features (funneling and smurfing-related) better inform anomaly detection than agnostic (graph rank and centrality) attributes alone."""
print(text)


Nevertheless, we started a new enrichment phase which added funneling and smurfing signaling features over all the April 2025 traffic, revealed a clear separation between anomaly and normal scores. As Table 4-10 shows, cases tied to these typologies began resulting in distinct score profiles. These behaviors were flagged from the enriched features capturing graph topology asymmetries (in degree vs. out degree) and temporal density (transaction bursts) around nodes.

Also, as shown in Table 4-11, the 1−WS similarity changed from 0.4646 to 0.4446 (WS distance from 0.5354 to 0.5554), supporting the intuition that business aware features (funneling and smurfing-related) better inform anomaly detection than agnostic (graph rank and centrality) attributes alone.


In [12]:

# Table 4-10 — Phase 2 stats
tbl_410 = pd.DataFrame([
    {"Period":"April 1st 2025",
     "Anomaly Rate": p2_apr1_stats.get("anomaly_rate"),
     "Anomaly Min Score": sci(p2_apr1_stats.get("anom_min")),
     "Anomaly Max Score": sci(p2_apr1_stats.get("anom_max")),
     "Normal Min Score":  sci(p2_apr1_stats.get("norm_min")),
     "Normal Max Score":  sci(p2_apr1_stats.get("norm_max"))},
    {"Period":"April 2025",
     "Anomaly Rate": p2_apr_stats.get("anomaly_rate"),
     "Anomaly Min Score": sci(p2_apr_stats.get("anom_min")),
     "Anomaly Max Score": sci(p2_apr_stats.get("anom_max")),
     "Normal Min Score":  sci(p2_apr_stats.get("norm_min")),
     "Normal Max Score":  sci(p2_apr_stats.get("norm_max"))},
])
print("Table 4-10 — Anomaly score separation after business-aware enrichment (Phase 2)")
print(tbl_410.to_string(index=False))
#tbl_410.to_csv("/mnt/data/Table_4_10_Phase2_global.csv", index=False)

# Table 4-11 — 1−WS stats
tbl_411 = pd.DataFrame([
    {"Period":"April 1st 2025",
     "1−WS (Phase 1)": 1.0 - p1_apr1_sim["ws"],
     "1−WS (Phase 2)": 1.0 - p2_apr1_sim["ws"],
     "WS Dist (Phase 1)": p1_apr1_sim["ws"],
     "WS Dist (Phase 2)": p2_apr1_sim["ws"],
     "Δ WS Dist (P2−P1)": p2_apr1_sim["ws"] - p1_apr1_sim["ws"]},
    {"Period":"April 2025",
     "1−WS (Phase 1)": 1.0 - p1_apr_sim["ws"],
     "1−WS (Phase 2)": 1.0 - p2_apr_sim["ws"],
     "WS Dist (Phase 1)": p1_apr_sim["ws"],
     "WS Dist (Phase 2)": p2_apr_sim["ws"],
     "Δ WS Dist (P2−P1)": p2_apr_sim["ws"] - p1_apr_sim["ws"]},
])
print("\nTable 4-11 — 1−WS similarity and WS distance after business-aware enrichment")
print(tbl_411.to_string(index=False))
#tbl_411.to_csv("/mnt/data/Table_4_11_1WS_comparison.csv", index=False)
#print("\nSaved: /mnt/data/Table_4_10_Phase2_global.csv and /mnt/data/Table_4_11_1WS_comparison.csv")


Table 4-10 — Anomaly score separation after business-aware enrichment (Phase 2)
        Period  Anomaly Rate Anomaly Min Score Anomaly Max Score Normal Min Score Normal Max Score
April 1st 2025      0.062016 -1.374533 × 10^-1 -4.825527 × 10^-6 1.901936 × 10^-5 2.693719 × 10^-1
    April 2025      0.051436 -1.921282 × 10^-1 -4.825527 × 10^-6 5.361697 × 10^-7 2.693719 × 10^-1

Table 4-11 — 1−WS similarity and WS distance after business-aware enrichment
        Period  1−WS (Phase 1)  1−WS (Phase 2)  WS Dist (Phase 1)  WS Dist (Phase 2)  Δ WS Dist (P2−P1)
April 1st 2025        0.409795        0.370421           0.590205           0.629579           0.039375
    April 2025        0.464608        0.444580           0.535392           0.555420           0.020028
